In [ ]:
class Station_point:
    def __init__(self, x, z, R, rho, q, m):
        G = 6.67408 *10**-11
        self.A = 4/3*np.pi*G*rho*R**3
        self.g = self.A*z**m/(x**2+z**2)**q
    def norm_g(self, g0):
        self.g0 = self.g/g0

def Measurement_generation(R,rho,m,q,z,Num_points):
    Centre_x = 0
    Centre_y = 0
    p0 = Station_point(0,z,R,rho,q,m)
    SD_x = 50
    
    Input_data_list_NN = [[0 for x in range (0,2)] for y in range(0,Num_points)]
    Station_points_list = [0 for x in range (0,Num_points)]

    for a in range (0,Num_points):
        Ran_x = np.abs(SD_x*np.random.randn())
        p = Station_point(Ran_x,z,R,rho,q,m)
        p.norm_g(p0.g)
        Input_data_list_NN[a][0] = Ran_x
        Input_data_list_NN[a][1] = p.g0
        Station_points_list[a] = p
    
    Max_x = 0
    for a in range (0,Num_points):
        if (np.max(Input_data_list_NN[a][0]) > Max_x):
            Max_x = np.max(Input_data_list_NN[a][0])
#     for a in range (0,Num_points):
#         Input_data_list_NN[a][0] = Input_data_list_NN[a][0]/Max_x 
    
    return [Input_data_list_NN,Max_x]     

# sample sample
Maxest_x = 0
Maxest_z = 0
SD_z = 150

Num_configs = 1000
Num_samples = 10

X_array_train = [0 for a in range(0,round(0.65*Num_configs))]
y_array_train = [0 for a in range(0,round(0.65*Num_configs))]

X_array_valid = [0 for a in range(0,round(0.15*Num_configs))]
y_array_valid = [0 for a in range(0,round(0.15*Num_configs))]

X_array_test = [0 for a in range(0,round(0.2*Num_configs))]
y_array_test = [0 for a in range(0,round(0.2*Num_configs))]

q_options = [1/2,1,3/2]
index = 0
for a in range(0,Num_configs):
    ran_z = np.abs(SD_z*np.random.rand())
    ran_q_index = np.round(np.abs(2*np.random.rand()))
    ran_q = q_options[np.int(ran_q_index)]
    R = 10
    rho = 2500
    m = 1
    if ran_q==1/2:
        m = -1
    [X,Max_x] = Measurement_generation(R,rho,m,ran_q,ran_z,Num_samples)
    if (Max_x > Maxest_x):
        Maxest_x = Max_x
    if (ran_z > Maxest_z):
        Maxest_z = ran_z
    
    y = [[0 for x in range (0,2)] for y in range(0,Num_samples)]
    for x in range(0,Num_samples):
        y[x][0] = ran_z
        y[x][1] = ran_q
    
    if (a < round(0.65*Num_configs)):
        X_array_train[a] = X
        y_array_train[a] = y
    elif (a < round(0.8*Num_configs)):
        X_array_valid[a-round(0.65*Num_configs)] = X
        y_array_valid[a-round(0.65*Num_configs)] = y 
    else:
        X_array_test[a-round(0.8*Num_configs)] = X
        y_array_test[a-round(0.8*Num_configs)] = y
    
for a in range(0,Num_configs):
    for b in range(0,Num_samples):
        if (a < round(0.65*Num_configs)):
            X_array_train[a][b][0] = X_array_train[a][b][0]/Maxest_x
            y_array_train[a][b][0] = y_array_train[a][b][0]/Maxest_z
        elif (a < round(0.8*Num_configs)):
            X_array_valid[a-round(0.65*Num_configs)][b][0] = X_array_valid[a-round(0.65*Num_configs)][b][0]/Maxest_x
            y_array_valid[a-round(0.65*Num_configs)][b][0] = y_array_valid[a-round(0.65*Num_configs)][b][0]/Maxest_z
        else:
            X_array_test[a-round(0.8*Num_configs)][b][0] = X_array_test[a-round(0.8*Num_configs)][b][0]/Maxest_x
            y_array_test[a-round(0.8*Num_configs)][b][0] = y_array_test[a-round(0.8*Num_configs)][b][0]/Maxest_z
              

    
# print(X_array)
# print(y_array)
# print(Maxest_x)

features_train = X_array_train
features_valid = X_array_valid
features_test = X_array_test
              
labels_train = y_array_train
labels_valid = y_array_valid
labels_test = y_array_test

# # Load the Features; list of Time of Flight (0th column), Departure time (1st column) and Arrival time (2nd column)
# # Time of Flight in seconds, Departure time in seconds since 1/1/2020 1:30 PM, Arrival time in seconds since 1/1/2020 1:30 PM
# features=np.loadtxt("Total_Features.txt")
# # Load the Labels; list of Delta V needed at Earth and Delta V needed at Mars
# # Optionally: sum them for a different label
# labels=np.loadtxt("Total_Labels.txt")